In [ ]:
import sys
sys.path.insert(0, '../')
from qhdopt import QHD
from sympy import symbols

In [ ]:
x, y, z = symbols('x y z')
f = (y**2) * z - x * z + (x**2) * y

In [ ]:
model = QHD.SymPy(f, [x, y, z])

In [ ]:
model.dwave_setup(8, api_key_from_file='../dwave_API_key')

In [ ]:
model.optimize(verbose = 1)

In [ ]:
model.qutip_setup(4, time_discretization=40)

In [ ]:
model.optimize(verbose = 1)

In [ ]:
model.ionq_setup(6, api_key_from_file='../ionq_API_key', time_discretization=5, shots = 1000)

In [ ]:
model.optimize(verbose = 1)